<a href="https://colab.research.google.com/github/espickle1/sequence-cleaning/blob/main/chimerax_color_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ChimeraX Color Script Generator

Upload per-residue entropy (or any scalar) values, sequences, and metadata.
This notebook generates a `.cxc` ChimeraX color script **per sequence** and
downloads them to your machine.

## 0. Setup – Clone repo & install dependencies

In [1]:
import os, subprocess

repo_dir = "sequence-cleaning"
if not os.path.isdir(repo_dir):
    subprocess.run(
        ["git", "clone", "https://github.com/espickle1/sequence-cleaning.git"],
        check=True,
    )

if os.path.basename(os.getcwd()) != repo_dir:
    os.chdir(repo_dir)

print(f"Working directory: {os.getcwd()}")

Working directory: /content/sequence-cleaning


In [3]:
import numpy as np
import pandas as pd
from google.colab import files
from analysis.chimerax_color_lib import (
    generate_chimerax_script,
    generate_chimerax_script_with_scalers,
    write_chimerax_script,
    fit_scaler,
    fit_minmax_scaler,
    scale_values_with_scaler,
    scale_values,
    compute_value_statistics,
    parse_range_string,
    create_value_mask,
    display_statistics_summary,
)

## 1. Upload files

Upload three CSV files:
- **Metadata file** – must contain `sequence_id` and `name` columns.
- **Sequences file** – must contain `sequence_id` and `sequence` columns.
- **Entropy file** – per-residue entropy values. Expected columns: `sequence_id` plus one or more value columns, **or** a single per-residue file (e.g. `residue_position, entropy, ...`) that applies to one sequence.


In [4]:
print("Upload the METADATA file (.csv):")
meta_upload = files.upload()
meta_filename = list(meta_upload.keys())[0]
df_metadata = pd.read_csv(meta_filename)
print(f"Loaded {meta_filename}: {df_metadata.shape}")
print(f"Columns: {list(df_metadata.columns)}")
display(df_metadata)

Upload the METADATA file (.csv):


Saving metadata.csv to metadata (1).csv
Loaded metadata (1).csv: (12, 10)
Columns: ['sequence_id', 'original_header', 'name', 'date', 'source_file', 'field_1', 'field_2', 'field_3', 'field_4', 'field_5']


,sequence_id,original_header,name,date,source_file,field_1,field_2,field_3,field_4,field_5
0,0ea58344488f,PB1|A / H5N1|A/Texas/37/2024||2.3.4.4b|2024-03...,PB1,2024-03-28,gisaid_epiflu_sequence (3).fasta,A / H5N1,A/Texas/37/2024,2.3.4.4b,EPI_ISL_19027114,EPI3171490
1,3bfaf386b5b9,PB1-F2|A / H5N1|A/Texas/37/2024||2.3.4.4b|2024...,PB1-F2,2024-03-28,gisaid_epiflu_sequence (3).fasta,A / H5N1,A/Texas/37/2024,2.3.4.4b,EPI_ISL_19027114,EPI3171490
2,301aaa17b0da,NS1|A / H5N1|A/Texas/37/2024||2.3.4.4b|2024-03...,NS1,2024-03-28,gisaid_epiflu_sequence (3).fasta,A / H5N1,A/Texas/37/2024,2.3.4.4b,EPI_ISL_19027114,EPI3171491
3,7c2779a1aec1,NEP|A / H5N1|A/Texas/37/2024||2.3.4.4b|2024-03...,NEP,2024-03-28,gisaid_epiflu_sequence (3).fasta,A / H5N1,A/Texas/37/2024,2.3.4.4b,EPI_ISL_19027114,EPI3171491
4,2e51a71e2288,PB2|A / H5N1|A/Texas/37/2024||2.3.4.4b|2024-03...,PB2,2024-03-28,gisaid_epiflu_sequence (3).fasta,A / H5N1,A/Texas/37/2024,2.3.4.4b,EPI_ISL_19027114,EPI3171492
5,a9003eeee5e9,M1|A / H5N1|A/Texas/37/2024||2.3.4.4b|2024-03-...,M1,2024-03-28,gisaid_epiflu_sequence (3).fasta,A / H5N1,A/Texas/37/2024,2.3.4.4b,EPI_ISL_19027114,EPI3171493
6,7b65cc89178f,M2|A / H5N1|A/Texas/37/2024||2.3.4.4b|2024-03-...,M2,2024-03-28,gisaid_epiflu_sequence (3).fasta,A / H5N1,A/Texas/37/2024,2.3.4.4b,EPI_ISL_19027114,EPI3171493
7,41343588f288,NA|A / H5N1|A/Texas/37/2024||2.3.4.4b|2024-03-...,NaN,2024-03-28,gisaid_epiflu_sequence (3).fasta,A / H5N1,A/Texas/37/2024,2.3.4.4b,EPI_ISL_19027114,EPI3171486
8,2d720887d514,PA|A / H5N1|A/Texas/37/2024||2.3.4.4b|2024-03-...,PA,2024-03-28,gisaid_epiflu_sequence (3).fasta,A / H5N1,A/Texas/37/2024,2.3.4.4b,EPI_ISL_19027114,EPI3171487
9,2a0b35718118,HA|A / H5N1|A/Texas/37/2024||2.3.4.4b|2024-03-...,HA,2024-03-28,gisaid_epiflu_sequence (3).fasta,A / H5N1,A/Texas/37/2024,2.3.4.4b,EPI_ISL_19027114,EPI3171488


In [5]:
print("Upload the SEQUENCES file (.csv):")
seq_upload = files.upload()
seq_filename = list(seq_upload.keys())[0]
df_sequences = pd.read_csv(seq_filename)
print(f"Loaded {seq_filename}: {df_sequences.shape}")
print(f"Columns: {list(df_sequences.columns)}")
display(df_sequences)

Upload the SEQUENCES file (.csv):


Saving sequences.csv to sequences (1).csv
Loaded sequences (1).csv: (12, 3)
Columns: ['sequence_id', 'sequence', 'length']


,sequence_id,sequence,length
0,0ea58344488f,MDVNPTLLFLKVPAQNAISTTFPYTGDPPYSHGTGTGYTMDTVNRT...,757
1,3bfaf386b5b9,MEQEQDIPWTQLTEHINIQKKGNGQQTQKPGHLNSIQLMDHCLMTM...,90
2,301aaa17b0da,MDSNTVLSFQVDCFLWHVRKRFADQELGDAPFLDRLRRDRKSLRGR...,230
3,7c2779a1aec1,MDSNTVLSFQDILMRMSKMQLGSSSEDLNGMITQFESLKLYRDSLG...,121
4,2e51a71e2288,MERIKELRDLMSQSRTREILTKTTVDHMAIIKKYTSGRQEKNPALR...,759
5,a9003eeee5e9,MSLLTEVETYVLSIVPSGPLKAEIAQRLEDVFAGKNTDLEALMEWL...,252
6,7b65cc89178f,MSLLTEVETPTKNGWECNCSDSSDPLVIAASIIGILHLILWILDRL...,97
7,41343588f288,MNPNQKITTIGSICMVIGIVSLMLQIGNIISIWVSHSIQTGNQYQP...,469
8,2d720887d514,MEDFVRQCFNPMIVELAEKAMKEYGEDPKIETNKFAAICTHLEVCF...,716
9,2a0b35718118,MENIVLLLAIVSLVKSDQICIGYHANNSTEQVDTIMEKNVTVTHAQ...,567


In [6]:
print("Upload the ENTROPY file (.csv):")
entropy_upload = files.upload()
entropy_filename = list(entropy_upload.keys())[0]
df_entropy = pd.read_csv(entropy_filename)
print(f"Loaded {entropy_filename}: {df_entropy.shape}")
print(f"Columns: {list(df_entropy.columns)}")
df_entropy.head()

Upload the ENTROPY file (.csv):


Saving entropy_per_residue_2e51a71e2288.csv to entropy_per_residue_2e51a71e2288.csv
Loaded entropy_per_residue_2e51a71e2288.csv: (761, 3)
Columns: ['residue_position', 'entropy', 'classification']


,residue_position,entropy,classification
0,1,2.437500,constrained
1,2,0.500000,constrained
2,3,2.640625,intermediate
3,4,2.765625,intermediate
4,5,2.750000,intermediate


## 1b. Multi-file normalization (optional)

Enable this to normalize entropy values across **multiple** entropy files. This ensures
all sequences use the same color scale, making comparisons meaningful.

- **Single file mode** (default): Each sequence is colored based on its own min/max values.
- **Multi-file mode**: All entropy values are combined to compute a shared scale, then each
  sequence is colored using that shared scale.

In [16]:
# --- Multi-file normalization setup ---
import re
from pathlib import Path

_multi = input("Use multi-file normalization? (y/n) [n]: ").strip().lower()
MULTI_FILE_MODE = _multi in ("y", "yes")

# Store all entropy data: list of dicts with 'seq_id', 'label', 'values', 'filename', 'model', 'chain'
all_entropy_data = []

if MULTI_FILE_MODE:
    print("\n--- Multi-file mode enabled ---")
    print("Upload additional entropy files. These will be combined with the first file")
    print("to compute a shared normalization scale.")
    print("You will specify model/chain IDs for each file.\n")

    # Add the first file that was already uploaded
    if "sequence_id" not in df_entropy.columns:
        # Per-residue format - extract seq_id from filename
        match = re.search(r"entropy_per_residue_(.+)\.csv", entropy_filename)
        first_seq_id = match.group(1) if match else Path(entropy_filename).stem

        # Get value column
        if "entropy" in df_entropy.columns:
            value_col = "entropy"
        else:
            numeric_cols = [c for c in df_entropy.select_dtypes(include="number").columns if c != "residue_position"]
            value_col = numeric_cols[0] if numeric_cols else df_entropy.columns[-1]

        # Look up name from metadata
        first_label = first_seq_id
        if "name" in df_metadata.columns and "sequence_id" in df_metadata.columns:
            name_match = df_metadata.loc[df_metadata["sequence_id"] == first_seq_id, "name"]
            if len(name_match) and pd.notna(name_match.iloc[0]):
                first_label = name_match.iloc[0]

        # Get model/chain for first file
        print(f"File 1: {entropy_filename} -> {first_label}")
        _model = input(f"  Model ID for '{first_label}' [1]: ").strip() or "1"
        _chain = input(f"  Chain ID for '{first_label}' (blank for none): ").strip()

        all_entropy_data.append({
            "seq_id": first_seq_id,
            "label": first_label,
            "values": df_entropy[value_col].values.astype(float),
            "filename": entropy_filename,
            "model": int(_model),
            "chain": _chain,
        })
        print(f"  -> Model #{_model}" + (f"/{_chain}" if _chain else "") + f", {len(df_entropy)} residues\n")
    else:
        raise ValueError("Multi-file mode currently only supports per-residue format (Format B) entropy files.")

    # Upload additional files
    while True:
        _more = input("Upload another entropy file? (y/n) [n]: ").strip().lower()
        if _more not in ("y", "yes"):
            break

        print("\nUpload the next ENTROPY file (.csv):")
        extra_upload = files.upload()
        extra_filename = list(extra_upload.keys())[0]
        df_extra = pd.read_csv(extra_filename)

        if "sequence_id" in df_extra.columns:
            print("Warning: File has sequence_id column. Multi-file mode expects per-residue files.")
            continue

        # Extract seq_id from filename
        match = re.search(r"entropy_per_residue_(.+)\.csv", extra_filename)
        extra_seq_id = match.group(1) if match else Path(extra_filename).stem

        # Get value column
        if "entropy" in df_extra.columns:
            extra_value_col = "entropy"
        else:
            numeric_cols = [c for c in df_extra.select_dtypes(include="number").columns if c != "residue_position"]
            extra_value_col = numeric_cols[0] if numeric_cols else df_extra.columns[-1]

        # Look up name
        extra_label = extra_seq_id
        if "name" in df_metadata.columns and "sequence_id" in df_metadata.columns:
            name_match = df_metadata.loc[df_metadata["sequence_id"] == extra_seq_id, "name"]
            if len(name_match) and pd.notna(name_match.iloc[0]):
                extra_label = name_match.iloc[0]

        # Get model/chain for this file
        file_num = len(all_entropy_data) + 1
        print(f"File {file_num}: {extra_filename} -> {extra_label}")
        _model = input(f"  Model ID for '{extra_label}' [{file_num}]: ").strip() or str(file_num)
        _chain = input(f"  Chain ID for '{extra_label}' (blank for none): ").strip()

        all_entropy_data.append({
            "seq_id": extra_seq_id,
            "label": extra_label,
            "values": df_extra[extra_value_col].values.astype(float),
            "filename": extra_filename,
            "model": int(_model),
            "chain": _chain,
        })
        print(f"  -> Model #{_model}" + (f"/{_chain}" if _chain else "") + f", {len(df_extra)} residues\n")

    print(f"\n=== Loaded {len(all_entropy_data)} entropy file(s) for multi-file normalization ===")
    for i, data in enumerate(all_entropy_data, 1):
        spec = f"#{data['model']}" + (f"/{data['chain']}" if data['chain'] else "")
        print(f"  {i}. {data['label']} ({data['seq_id']}): {len(data['values'])} residues, {spec}")
else:
    print("Single-file mode: each sequence normalized independently.")

Use multi-file normalization? (y/n) [n]: y

--- Multi-file mode enabled ---
Upload additional entropy files. These will be combined with the first file
to compute a shared normalization scale.
You will specify model/chain IDs for each file.

File 1: entropy_per_residue_2e51a71e2288.csv -> PB2
  Model ID for 'PB2' [1]: 1
  Chain ID for 'PB2' (blank for none): b
  -> Model #1/b, 761 residues

Upload another entropy file? (y/n) [n]: y

Upload the next ENTROPY file (.csv):


Saving entropy_per_residue_2d720887d514.csv to entropy_per_residue_2d720887d514.csv
File 2: entropy_per_residue_2d720887d514.csv -> PA
  Model ID for 'PA' [2]: 1
  Chain ID for 'PA' (blank for none): c
  -> Model #1/c, 718 residues

Upload another entropy file? (y/n) [n]: y

Upload the next ENTROPY file (.csv):


Saving entropy_per_residue_0ea58344488f.csv to entropy_per_residue_0ea58344488f.csv
File 3: entropy_per_residue_0ea58344488f.csv -> PB1
  Model ID for 'PB1' [3]: 1
  Chain ID for 'PB1' (blank for none): a
  -> Model #1/a, 759 residues

Upload another entropy file? (y/n) [n]: n

=== Loaded 3 entropy file(s) for multi-file normalization ===
  1. PB2 (2e51a71e2288): 761 residues, #1/b
  2. PA (2d720887d514): 718 residues, #1/c
  3. PB1 (0ea58344488f): 759 residues, #1/a


## 2. Merge files on `sequence_id`

In [17]:
# Detect entropy file format and build a unified structure
#
# Format A: Wide table with sequence_id + value columns (one row per sequence)
# Format B: Per-residue table (residue_position, entropy, ...) for a single sequence
#           In this case sequence_id is extracted from the filename.

if "sequence_id" in df_entropy.columns:
    # Format A -- entropy file already has sequence_id
    available_ids = df_entropy["sequence_id"].unique().tolist()
    print(f"Available sequence_ids ({len(available_ids)}):")
    for sid in available_ids:
        print(f"  - {sid}")

    print("\nEnter sequence_id(s) to process (comma-separated), or 'all' for all:")
    selection = input("sequence_id(s) [all]: ").strip() or "all"

    if selection.lower() == "all":
        selected_ids = available_ids
    else:
        selected_ids = [s.strip() for s in selection.split(",") if s.strip()]
        # Validate selections
        invalid = [s for s in selected_ids if s not in available_ids]
        if invalid:
            print(f"Warning: unknown sequence_id(s) ignored: {invalid}")
        selected_ids = [s for s in selected_ids if s in available_ids]

    if not selected_ids:
        raise ValueError("No valid sequence_id selected.")

    print(f"\nSelected {len(selected_ids)} sequence(s): {selected_ids}")

    # Filter entropy data to selected IDs
    df_entropy_filtered = df_entropy[df_entropy["sequence_id"].isin(selected_ids)]

    merge_cols = ["sequence_id", "sequence"]
    if "sequence" in df_sequences.columns:
        df_merged = df_entropy_filtered.merge(
            df_sequences[merge_cols], on="sequence_id", how="inner"
        )
    else:
        df_merged = df_entropy_filtered.copy()

    if "name" in df_metadata.columns:
        df_merged = df_merged.merge(
            df_metadata[["sequence_id", "name"]], on="sequence_id", how="left"
        )

    value_columns = [
        c for c in df_entropy.columns if c != "sequence_id"
    ]
    print(f"Format A detected (wide table). Value columns: {value_columns}")
    print(f"Merged rows: {len(df_merged)}")
    display(df_merged.head())
    ENTROPY_FORMAT = "wide"

else:
    # Format B -- per-residue file without sequence_id
    # Try to extract sequence_id from the entropy filename
    # Pipeline exports files named like: entropy_per_residue_{seq_id}.csv
    import re
    match = re.search(r"entropy_per_residue_(.+)\.csv", entropy_filename)
    if match:
        inferred_seq_id = match.group(1)
    else:
        inferred_seq_id = Path(entropy_filename).stem

    print(f"Format B detected (per-residue). Inferred sequence_id: {inferred_seq_id}")
    override = input(f"Use this sequence_id? (press Enter to accept, or type a new one): ").strip()
    if override:
        inferred_seq_id = override
        print(f"Using sequence_id: {inferred_seq_id}")

    # Determine which column holds the values
    if "entropy" in df_entropy.columns:
        value_col = "entropy"
    else:
        # Use the first numeric column that isn't residue_position
        numeric_cols = [
            c for c in df_entropy.select_dtypes(include="number").columns
            if c != "residue_position"
        ]
        value_col = numeric_cols[0] if numeric_cols else df_entropy.columns[-1]

    print(f"Using value column: '{value_col}'")
    print(f"Residues: {len(df_entropy)}")

    # Look up the name from metadata
    label = inferred_seq_id
    if "name" in df_metadata.columns and "sequence_id" in df_metadata.columns:
        name_match = df_metadata.loc[
            df_metadata["sequence_id"] == inferred_seq_id, "name"
        ]
        if len(name_match):
            label = name_match.iloc[0]

    # Store for the generation step
    df_merged = None
    per_residue_info = {
        "seq_id": inferred_seq_id,
        "label": label,
        "values": df_entropy[value_col].values.astype(float),
    }
    value_columns = None
    ENTROPY_FORMAT = "per_residue"
    display(df_entropy.head())

Format B detected (per-residue). Inferred sequence_id: 2e51a71e2288
Use this sequence_id? (press Enter to accept, or type a new one): 
Using value column: 'entropy'
Residues: 761


,residue_position,entropy,classification
0,1,2.437500,constrained
1,2,0.500000,constrained
2,3,2.640625,intermediate
3,4,2.765625,intermediate
4,5,2.750000,intermediate


## 3. Configure color mapping

Adjust these parameters as needed before generating the scripts.

In [18]:
# --- Interactive Configuration ---

# Shared settings (apply to all files)
if ENTROPY_FORMAT == "wide" and value_columns:
    print(f"Value columns: {value_columns}")

CMAP_NAME = input("Colormap name [Greys]: ").strip() or "Greys"

print("Transform methods: none, log, minmax, quantile, power, standard, robust")
TRANSFORM_METHOD = input("Transform method [none]: ").strip() or "none"

_color = input("Enable color mapping? (y/n) [y]: ").strip().lower()
COLOR = _color not in ("n", "no")

COLOR_INVERT = False
if COLOR:
    _ci = input("Invert colormap? (y/n) [n]: ").strip().lower()
    COLOR_INVERT = _ci in ("y", "yes")

_trans = input("Enable transparency mapping? (y/n) [n]: ").strip().lower()
TRANSPARENCY = _trans in ("y", "yes")

TRANSPARENCY_INVERT = False
if TRANSPARENCY:
    _ti = input("Invert transparency? (y/n) [n]: ").strip().lower()
    TRANSPARENCY_INVERT = _ti in ("y", "yes")

# Model/chain configuration depends on mode
if MULTI_FILE_MODE and all_entropy_data:
    # Multi-file mode: model/chain already set per file, allow editing
    print("\n--- Per-file Model/Chain Configuration ---")
    print("Current assignments:")
    for i, data in enumerate(all_entropy_data):
        spec = f"#{data['model']}" + (f"/{data['chain']}" if data['chain'] else "")
        print(f"  {i+1}. {data['label']}: {spec}")

    _edit = input("\nEdit model/chain assignments? (y/n) [n]: ").strip().lower()
    if _edit in ("y", "yes"):
        for i, data in enumerate(all_entropy_data):
            print(f"\n{data['label']} (currently #{data['model']}" + (f"/{data['chain']}" if data['chain'] else "") + "):")
            _model = input(f"  Model ID [{data['model']}]: ").strip()
            if _model:
                data['model'] = int(_model)
            _chain = input(f"  Chain ID [{data['chain'] or 'none'}]: ").strip()
            if _chain.lower() == 'none':
                data['chain'] = ""
            elif _chain:
                data['chain'] = _chain

        print("\nUpdated assignments:")
        for i, data in enumerate(all_entropy_data):
            spec = f"#{data['model']}" + (f"/{data['chain']}" if data['chain'] else "")
            print(f"  {i+1}. {data['label']}: {spec}")

    # These are not used in multi-file mode but define for consistency
    MODEL = None
    CHAIN = None
else:
    # Single-file mode: one model/chain for all
    _model = input("Model ID [1]: ").strip() or "1"
    MODEL = int(_model)
    CHAIN = input("Chain ID (e.g. A, B — leave blank for none): ").strip()

# Summary
print(f"\n=== Configuration Summary ===")
print(f"Colormap: {CMAP_NAME}, Transform: {TRANSFORM_METHOD}")
print(f"Color: {COLOR} (invert={COLOR_INVERT}), Transparency: {TRANSPARENCY} (invert={TRANSPARENCY_INVERT})")

if MULTI_FILE_MODE and all_entropy_data:
    print("Mode: Multi-file normalization")
    for data in all_entropy_data:
        spec = f"#{data['model']}" + (f"/{data['chain']}" if data['chain'] else "")
        print(f"  - {data['label']}: {spec}")
else:
    print(f"Mode: Single-file")
    print(f"Model: #{MODEL}" + (f"/{CHAIN}" if CHAIN else ""))

Colormap name [Greys]: RdBu
Transform methods: none, log, minmax, quantile, power, standard, robust
Transform method [none]: quantile
Enable color mapping? (y/n) [y]: y
Invert colormap? (y/n) [n]: y
Enable transparency mapping? (y/n) [n]: n

--- Per-file Model/Chain Configuration ---
Current assignments:
  1. PB2: #1/b
  2. PA: #1/c
  3. PB1: #1/a

Edit model/chain assignments? (y/n) [n]: n

=== Configuration Summary ===
Colormap: RdBu, Transform: quantile
Color: True (invert=True), Transparency: False (invert=False)
Mode: Multi-file normalization
  - PB2: #1/b
  - PA: #1/c
  - PB1: #1/a


## 3b. Configure value cutoffs (optional)

After transformation, you can filter which residues are included in the CXC output
based on their transformed values.

**For quantile (rank-based) transforms:**
- Values are ordinal ranks from 0 to 1
- Specify rank ranges like `0-0.25` (bottom quartile) or `0.75-1.0` (top quartile)
- Multiple ranges: `0-0.1, 0.9-1.0` (extreme 10% on both ends)

**For other transforms (log, power, standard, robust, minmax, none):**
- Values are continuous (rational numbers)
- Statistics (mean, std, min, max) are displayed to help choose cutoffs
- Specify value ranges based on the transformed values

In [19]:
# --- Value Cutoff Configuration ---
# This cell computes statistics on transformed values and allows filtering

# Dictionary to store masks: key = seq_id or index, value = boolean mask array
residue_masks = {}
USE_CUTOFFS = False

_use_cutoffs = input("Apply value cutoffs to filter residues? (y/n) [n]: ").strip().lower()
USE_CUTOFFS = _use_cutoffs in ("y", "yes")

if USE_CUTOFFS:
    is_rank_based = TRANSFORM_METHOD == "quantile"

    if is_rank_based:
        print("\n" + "="*60)
        print("QUANTILE TRANSFORM (Rank-based)")
        print("="*60)
        print("Values are ordinal ranks from 0 (lowest) to 1 (highest).")
        print("\nExamples of rank range specifications:")
        print("  0-0.25        : Bottom quartile (lowest 25%)")
        print("  0.75-1.0      : Top quartile (highest 25%)")
        print("  0-0.1, 0.9-1.0: Extreme 10% on both ends")
        print("  0.25-0.75     : Middle 50% (interquartile range)")
        print("  all           : Include all residues (no filtering)")
    else:
        print("\n" + "="*60)
        print(f"TRANSFORM: {TRANSFORM_METHOD.upper() if TRANSFORM_METHOD != 'none' else 'NONE (raw values)'}")
        print("="*60)
        print("Values are continuous. Statistics are shown to help choose cutoffs.")
        print("\nExamples of value range specifications:")
        print("  -1.5-0.5      : Values between -1.5 and 0.5")
        print("  0-2, 5-10     : Multiple ranges (0 to 2 OR 5 to 10)")
        print("  all           : Include all residues (no filtering)")

    # --- Multi-file mode ---
    if MULTI_FILE_MODE and all_entropy_data:
        # Fit transform scaler on combined data for consistent statistics
        combined_values = np.concatenate([d["values"] for d in all_entropy_data])
        transform_scaler_for_stats = fit_scaler(combined_values, method=TRANSFORM_METHOD)

        print("\n" + "-"*60)
        print("COMBINED STATISTICS (all files)")
        print("-"*60)
        combined_stats = compute_value_statistics(
            combined_values,
            transform_method=TRANSFORM_METHOD,
            transform_scaler=transform_scaler_for_stats,
        )
        combined_stats["transform_method"] = TRANSFORM_METHOD
        print(display_statistics_summary(combined_stats, label="All Files Combined"))

        # Show per-file statistics
        print("\n" + "-"*60)
        print("PER-FILE STATISTICS")
        print("-"*60)
        for i, data in enumerate(all_entropy_data):
            stats = compute_value_statistics(
                data["values"],
                transform_method=TRANSFORM_METHOD,
                transform_scaler=transform_scaler_for_stats,
            )
            stats["transform_method"] = TRANSFORM_METHOD
            print(f"\n{i+1}. {data['label']} ({data['seq_id']}):")
            print(f"   Residues: {stats['n_residues']}")
            print(f"   Range: {stats['min']:.4f} - {stats['max']:.4f}")
            print(f"   Mean: {stats['mean']:.4f}, Std: {stats['std']:.4f}")
            # Store transformed values for filtering
            data["transformed_values"] = stats["transformed_values"]

        # Ask for range specification
        print("\n" + "-"*60)
        print("SPECIFY CUTOFF RANGES")
        print("-"*60)

        _same_range = input("Use same range for all files? (y/n) [y]: ").strip().lower()
        use_same_range = _same_range not in ("n", "no")

        if use_same_range:
            if is_rank_based:
                range_str = input("Rank range(s) for all files [all]: ").strip() or "all"
            else:
                range_str = input("Value range(s) for all files [all]: ").strip() or "all"

            try:
                ranges = parse_range_string(range_str)
                if ranges:
                    print(f"\nApplying ranges: {ranges}")
                    for data in all_entropy_data:
                        mask = create_value_mask(data["transformed_values"], ranges)
                        residue_masks[data["seq_id"]] = mask
                        included = np.sum(mask)
                        print(f"  {data['label']}: {included}/{len(mask)} residues included ({100*included/len(mask):.1f}%)")
                else:
                    print("\nNo filtering - all residues will be included.")
            except ValueError as e:
                print(f"Error parsing range: {e}")
                print("No filtering will be applied.")
        else:
            # Per-file range specification
            for data in all_entropy_data:
                print(f"\n{data['label']} ({data['seq_id']}):")
                if is_rank_based:
                    range_str = input(f"  Rank range(s) [all]: ").strip() or "all"
                else:
                    range_str = input(f"  Value range(s) [all]: ").strip() or "all"

                try:
                    ranges = parse_range_string(range_str)
                    if ranges:
                        mask = create_value_mask(data["transformed_values"], ranges)
                        residue_masks[data["seq_id"]] = mask
                        included = np.sum(mask)
                        print(f"    -> {included}/{len(mask)} residues included ({100*included/len(mask):.1f}%)")
                    else:
                        print(f"    -> All residues included (no filtering)")
                except ValueError as e:
                    print(f"    Error: {e}. No filtering for this file.")

    # --- Single-file mode: per-residue format ---
    elif ENTROPY_FORMAT == "per_residue":
        values = per_residue_info["values"]
        stats = compute_value_statistics(values, transform_method=TRANSFORM_METHOD)
        stats["transform_method"] = TRANSFORM_METHOD
        print("\n" + display_statistics_summary(stats, label=per_residue_info["label"]))

        # Store transformed values
        per_residue_info["transformed_values"] = stats["transformed_values"]

        print("\n" + "-"*60)
        if is_rank_based:
            range_str = input("Rank range(s) to include [all]: ").strip() or "all"
        else:
            range_str = input("Value range(s) to include [all]: ").strip() or "all"

        try:
            ranges = parse_range_string(range_str)
            if ranges:
                mask = create_value_mask(stats["transformed_values"], ranges)
                residue_masks[per_residue_info["seq_id"]] = mask
                included = np.sum(mask)
                print(f"-> {included}/{len(mask)} residues included ({100*included/len(mask):.1f}%)")
            else:
                print("-> All residues included (no filtering)")
        except ValueError as e:
            print(f"Error parsing range: {e}")
            print("No filtering will be applied.")

    # --- Single-file mode: wide format ---
    elif ENTROPY_FORMAT == "wide":
        print("\nComputing statistics for each sequence...")

        for _, row in df_merged.iterrows():
            seq_id = row["sequence_id"]
            label = row.get("name", seq_id) or seq_id
            values = row[value_columns].values.astype(float)

            stats = compute_value_statistics(values, transform_method=TRANSFORM_METHOD)
            stats["transform_method"] = TRANSFORM_METHOD
            print(f"\n{label} ({seq_id}):")
            print(f"  Residues: {stats['n_residues']}")
            print(f"  Range: {stats['min']:.4f} - {stats['max']:.4f}")
            print(f"  Mean: {stats['mean']:.4f}, Std: {stats['std']:.4f}")

        print("\n" + "-"*60)
        _same_range = input("Use same range for all sequences? (y/n) [y]: ").strip().lower()
        use_same_range = _same_range not in ("n", "no")

        if use_same_range:
            if is_rank_based:
                range_str = input("Rank range(s) for all sequences [all]: ").strip() or "all"
            else:
                range_str = input("Value range(s) for all sequences [all]: ").strip() or "all"

            try:
                ranges = parse_range_string(range_str)
                if ranges:
                    print(f"\nApplying ranges: {ranges}")
                    for _, row in df_merged.iterrows():
                        seq_id = row["sequence_id"]
                        values = row[value_columns].values.astype(float)
                        transformed = scale_values(values, method=TRANSFORM_METHOD)
                        mask = create_value_mask(transformed, ranges)
                        residue_masks[seq_id] = mask
                        included = np.sum(mask)
                        print(f"  {seq_id}: {included}/{len(mask)} residues ({100*included/len(mask):.1f}%)")
                else:
                    print("\nNo filtering - all residues will be included.")
            except ValueError as e:
                print(f"Error parsing range: {e}")
        else:
            for _, row in df_merged.iterrows():
                seq_id = row["sequence_id"]
                label = row.get("name", seq_id) or seq_id
                values = row[value_columns].values.astype(float)

                print(f"\n{label} ({seq_id}):")
                if is_rank_based:
                    range_str = input(f"  Rank range(s) [all]: ").strip() or "all"
                else:
                    range_str = input(f"  Value range(s) [all]: ").strip() or "all"

                try:
                    ranges = parse_range_string(range_str)
                    if ranges:
                        transformed = scale_values(values, method=TRANSFORM_METHOD)
                        mask = create_value_mask(transformed, ranges)
                        residue_masks[seq_id] = mask
                        included = np.sum(mask)
                        print(f"    -> {included}/{len(mask)} residues ({100*included/len(mask):.1f}%)")
                    else:
                        print(f"    -> All residues included")
                except ValueError as e:
                    print(f"    Error: {e}")

    # Summary
    if residue_masks:
        print("\n" + "="*60)
        print("CUTOFF SUMMARY")
        print("="*60)
        total_residues = sum(len(m) for m in residue_masks.values())
        total_included = sum(np.sum(m) for m in residue_masks.values())
        print(f"Total: {total_included}/{total_residues} residues will be included ({100*total_included/total_residues:.1f}%)")
    else:
        print("\nNo cutoffs applied - all residues will be included.")
else:
    print("Cutoffs disabled - all residues will be included in CXC output.")

Apply value cutoffs to filter residues? (y/n) [n]: y

QUANTILE TRANSFORM (Rank-based)
Values are ordinal ranks from 0 (lowest) to 1 (highest).

Examples of rank range specifications:
  0-0.25        : Bottom quartile (lowest 25%)
  0.75-1.0      : Top quartile (highest 25%)
  0-0.1, 0.9-1.0: Extreme 10% on both ends
  0.25-0.75     : Middle 50% (interquartile range)
  all           : Include all residues (no filtering)

------------------------------------------------------------
COMBINED STATISTICS (all files)
------------------------------------------------------------
=== Statistics for All Files Combined ===
Transform type: Quantile (rank-based, ordinal values 0-1)
  Residues: 2238
  Rank range: 0.0000 - 1.0000
  Quartiles:
    Q1 (25%): 0.2633
    Q2 (50%): 0.5435
    Q3 (75%): 0.7477

  Suggested rank ranges:
    Bottom quartile: 0-0.25
    Middle 50%: 0.25-0.75
    Top quartile: 0.75-1.0

------------------------------------------------------------
PER-FILE STATISTICS
----------

## 4. Generate `.cxc` scripts (one per sequence)

In [20]:
import os
from pathlib import Path

output_dir = "cxc_output"
os.makedirs(output_dir, exist_ok=True)

generated_files = []

def _make_cxc(values, label, seq_id, model, chain, mask=None):
    """Generate a .cxc file for one sequence (single-file mode)."""
    script = generate_chimerax_script(
        values,
        cmap_name=CMAP_NAME,
        transform_method=TRANSFORM_METHOD,
        color=COLOR,
        color_invert=COLOR_INVERT,
        transparency=TRANSPARENCY,
        transparency_invert=TRANSPARENCY_INVERT,
        model=model,
        chain=chain,
        residue_mask=mask,
    )
    safe_label = str(label).replace(" ", "_").replace("/", "_")
    chain_str = chain if chain else ""
    cmap_str = f"{CMAP_NAME}_i" if COLOR_INVERT else CMAP_NAME
    # Add "filtered" suffix if mask is applied
    filter_str = "_filtered" if mask is not None and not np.all(mask) else ""
    out_path = os.path.join(output_dir, f"{safe_label}_{seq_id}_{model}{chain_str}_{cmap_str}_{TRANSFORM_METHOD}{filter_str}.cxc")
    write_chimerax_script(script, out_path)
    return out_path

def _make_cxc_with_scalers(values, label, seq_id, model, chain, transform_scaler, minmax_scaler, mask=None):
    """Generate a .cxc file using pre-fitted scalers (multi-file mode)."""
    script = generate_chimerax_script_with_scalers(
        values,
        transform_scaler=transform_scaler,
        minmax_scaler=minmax_scaler,
        cmap_name=CMAP_NAME,
        color=COLOR,
        color_invert=COLOR_INVERT,
        transparency=TRANSPARENCY,
        transparency_invert=TRANSPARENCY_INVERT,
        model=model,
        chain=chain,
        residue_mask=mask,
    )
    safe_label = str(label).replace(" ", "_").replace("/", "_")
    chain_str = chain if chain else ""
    cmap_str = f"{CMAP_NAME}_i" if COLOR_INVERT else CMAP_NAME
    # Add "filtered" suffix if mask is applied
    filter_str = "_filtered" if mask is not None and not np.all(mask) else ""
    out_path = os.path.join(output_dir, f"{safe_label}_{seq_id}_{model}{chain_str}_{cmap_str}_{TRANSFORM_METHOD}_multifile{filter_str}.cxc")
    write_chimerax_script(script, out_path)
    return out_path

# --- Multi-file mode: fit scalers on combined data ---
if MULTI_FILE_MODE and all_entropy_data:
    print("=== Multi-file normalization ===")

    # Combine all values
    combined_values = np.concatenate([d["values"] for d in all_entropy_data])
    print(f"Combined {len(all_entropy_data)} files: {len(combined_values)} total residues")
    print(f"  Min: {combined_values.min():.4f}, Max: {combined_values.max():.4f}")
    print(f"  Mean: {combined_values.mean():.4f}, Std: {combined_values.std():.4f}")

    # Fit transform scaler on combined data
    transform_scaler = fit_scaler(combined_values, method=TRANSFORM_METHOD)

    # Apply transform to combined data, then fit minmax scaler
    if transform_scaler is not None:
        scaled_combined = scale_values_with_scaler(combined_values, transform_scaler)
    else:
        scaled_combined = combined_values

    minmax_scaler = fit_minmax_scaler(scaled_combined)
    print(f"Fitted scalers on combined data (transform={TRANSFORM_METHOD})")

    if residue_masks:
        print(f"Applying cutoff filters to {len(residue_masks)} file(s)\n")
    else:
        print("No cutoff filters applied\n")

    # Generate .cxc for each sequence using the shared scalers and per-file model/chain
    for data in all_entropy_data:
        # Get mask for this sequence if available
        mask = residue_masks.get(data["seq_id"], None)

        out_path = _make_cxc_with_scalers(
            data["values"],
            data["label"],
            data["seq_id"],
            data["model"],
            data["chain"],
            transform_scaler,
            minmax_scaler,
            mask=mask,
        )
        generated_files.append(out_path)
        spec = f"#{data['model']}" + (f"/{data['chain']}" if data['chain'] else "")
        if mask is not None and not np.all(mask):
            included = np.sum(mask)
            print(f"  Created: {out_path} ({spec}) [{included}/{len(mask)} residues]")
        else:
            print(f"  Created: {out_path} ({spec})")

# --- Single-file mode ---
elif ENTROPY_FORMAT == "wide":
    # One row per sequence -- iterate over df_merged
    for _, row in df_merged.iterrows():
        seq_id = row["sequence_id"]
        label = row.get("name", seq_id) or seq_id
        values = row[value_columns].values.astype(float)

        # Get mask for this sequence if available
        mask = residue_masks.get(seq_id, None)

        out_path = _make_cxc(values, label, seq_id, MODEL, CHAIN, mask=mask)
        generated_files.append(out_path)
        if mask is not None and not np.all(mask):
            included = np.sum(mask)
            print(f"  Created: {out_path} [{included}/{len(mask)} residues]")
        else:
            print(f"  Created: {out_path}")
else:
    # Per-residue format -- single sequence
    # Get mask for this sequence if available
    mask = residue_masks.get(per_residue_info["seq_id"], None)

    out_path = _make_cxc(
        per_residue_info["values"],
        per_residue_info["label"],
        per_residue_info["seq_id"],
        MODEL,
        CHAIN,
        mask=mask,
    )
    generated_files.append(out_path)
    if mask is not None and not np.all(mask):
        included = np.sum(mask)
        print(f"  Created: {out_path} [{included}/{len(mask)} residues]")
    else:
        print(f"  Created: {out_path}")

print(f"\nGenerated {len(generated_files)} .cxc file(s).")

=== Multi-file normalization ===
Combined 3 files: 2238 total residues
  Min: 0.0114, Max: 2.9531
  Mean: 2.7674, Std: 0.2586
Fitted scalers on combined data (transform=quantile)
Applying cutoff filters to 3 file(s)

  Created: cxc_output/PB2_2e51a71e2288_1b_RdBu_i_quantile_multifile_filtered.cxc (#1/b) [22/761 residues]
  Created: cxc_output/PA_2d720887d514_1c_RdBu_i_quantile_multifile_filtered.cxc (#1/c) [75/718 residues]
  Created: cxc_output/PB1_0ea58344488f_1a_RdBu_i_quantile_multifile_filtered.cxc (#1/a) [78/759 residues]

Generated 3 .cxc file(s).


## 5. Download `.cxc` files

In [23]:
for f in generated_files:
    files.download(f)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>